In [23]:
import glob
import numpy as np
import os
import re
import pandas as pd
import os.path
import xml.etree.ElementTree as ET
import csv
import getopt
import sys
import json
import copy
from random import shuffle
from sklearn import utils
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
import keras
import keras.preprocessing.text as kpt
import nltk
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, LSTM, Input, Embedding
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, concatenate, Activation, Average
from keras.models import Model
from keras import optimizers
from keras.callbacks import CSVLogger
from nltk.tokenize import TreebankWordTokenizer

In [7]:
tree = ET.parse("C:\\Users\\morzm\\jup_txts\\test\\test1.xml")
root = tree.getroot()
n = 0

save_path = "C:\\Users\\morzm\\jup_txts\\corpus\\test\\"

for tweets in root.find('documents'):
    tweet = tweets.text
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
    line_to_write = text + '\n'
    n += 1
    with open(os.path.join(save_path,"output"+str(n)+".txt"), 'w', encoding="utf-8") as f:
        f.write(line_to_write)

In [8]:
xmls_directory = "C:\\Users\\morzm\\jup_txts\\text\\" # This is the path to the .xml files of the PAN2018 twitter corpus

truth_path = "C:\\Users\\morzm\\jup_txts\\en.txt" # This is the path to the .txt file containing the ids and genders of each twitter user

In [9]:
# This bit of code transforms the .xml files containing the tweets
# on the base of a truth file into .txts, depending on gender
# thus creating a corpus

with open(truth_path, 'r') as f:
    for line in f.readlines():
        string = line.partition(":::") # This line divides the author_id from the gender of the author
        if "male\n" in string: # this will ignore the last line of the list, but was the best solution I could come up with
            save_path = "C:\\Users\\morzm\\jup_txts\\corpus\\male\\"
            author_id = string[0]
            for file in os.listdir(xmls_directory):
                if file.endswith(author_id+".xml"):
                    os.chdir("C:\\Users\\morzm\\jup_txts\\text\\")
                    xml_file = file # so that the file can be parsed by ElementTree
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                    n = 0
                    
                    for tweets in root.find('documents'):
                        tweet = tweets.text
                        text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
                        line_to_write = text + '\n'
                        n += 1
                        with open(os.path.join(save_path, author_id+str(n)+".txt"), 'a', encoding="utf-8") as f:
                            f.write(line_to_write)
                              
        elif "female\n" in string: # same for female authors
            save_path = "C:\\Users\\morzm\\jup_txts\\corpus\\female\\"
            author_id = string[0]
            for file in os.listdir(xmls_directory):
                if file.endswith(author_id+".xml"):
                    os.chdir("C:\\Users\\morzm\\jup_txts\\text\\")
                    xml_file = file
                    tree = ET.parse(xml_file)
                    root = tree.getroot()
                    n = 0
                    
                    for tweets in root.find('documents'):
                        tweet = tweets.text
                        text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', tweet)
                        line_to_write = text + '\n'
                        n += 1
                        with open(os.path.join(save_path, author_id+str(n)+".txt"), 'a', encoding="utf-8") as f:
                            f.write(line_to_write)

In [3]:
def pre_process_data(filepath):
    male_path = os.path.join(filepath, 'male')
    female_path = os.path.join(filepath, 'female')
    male_label = 1
    female_label = 0
    dataset = []
    for filename in glob.glob(os.path.join(male_path, '*.txt')):
        with open(filename, 'r', encoding="utf-8") as f:
            dataset.append((male_label, f.read()))
    for filename in glob.glob(os.path.join(female_path, '*.txt')):
        with open(filename, 'r', encoding="utf-8") as f:
            dataset.append((female_label, f.read()))
    shuffle(dataset)
    return dataset

def collect_expected(dataset):
    return [sample[0] for sample in dataset]

In [4]:
def stem(word):
    regexp = r'^(.*?)(ing|ious|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

In [20]:
FLAGS = re.MULTILINE | re.DOTALL

def tokenize(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)

    # text = re_sub(r"#\S+", "hashtag")
    # text = re_sub(r"([!?.]){2,}", r" \1 ")
    # text = re_sub(r"\b(\S*?)(.)\2{2,}\b", r"\1\2 ")
    # text = re_sub(r"([A-Z]){2,}", "allcaps")
    # text = re_sub(r'([\w!.,?();*\[\]":\”\“])([!.,?();*\[\]":\”\“])', r'\1 \2')
    # text = re_sub(r'([!.,?();*:\[\]":\”\“])([\w!.,?();*\[\]":\”\“])', r'\1 \2')
    # text = re_sub(r'(.)(<)', r'\1 \2')
    # text = re_sub(r'(>)(.)', r'\1 \2')
    # text = re_sub(r'[\'\`\’\‘]', r'')
    # text = re_sub(r'\\n', r' ')
    # text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", " ")
    # text = re_sub(r"#(\S+)", r" ") # replace #name with name
    text = re_sub(r"(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))", " em_positive ")  # Smile -- :), : ), :-), (:, ( :, (-:, :')
    text = re_sub(r"(:\s?D|:-D|x-?D|X-?D)", " em_positive ")  # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    text = re_sub(r"(<3|:\*)", " em_positive ")  # Love -- <3, :*
    text = re_sub(r"(;-?\)|;-?D|\(-?;)", " em_positive ")  # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    text = re_sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', " em_negative ")  # Sad -- :-(, : (, :(, ):, )-:
    text = re_sub(r'(:,\(|:\'\(|:"\()', " em_negative ")  # Cry -- :,(, :'(, :"(
    text = re_sub(r"(-|\')", "")  # remove &
    text = re_sub(r"/", " / ")
    text = re_sub(r"@[0-9]+-", " ")
    text = re_sub(r"@\w+", " ")
    text = re_sub(r"{}{}[)dD]+|[)dD]+{}{}".format(eyes, nose, nose, eyes), " em_positive ")
    text = re_sub(r"{}{}p+".format(eyes, nose), " em_positive ")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), " em_negative ")
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), " em_neutralface ")
    text = re_sub(r"<3", " heart ")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " ")
    text = re_sub(r'-', r' ')
    # text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "  ")
    #     text = re_sub(r"([pls?s]){2,}", r"\1")
    #     text = re_sub(r"([plz?z]){2,}", r"\1")
    text = re_sub(r'\\n', r' ')
    text = re_sub(r" app ", " application ")
    text = re_sub(r"app", " application")
    text = re_sub(r" wil ", " will ")
    text = re_sub(r" im ", " i am ")
    text = re_sub(r" al ", " all ")
    # text = re_sub(r"<3","love")
    text = re_sub(r" sx ", " sex ")
    text = re_sub(r" u ", " you ")
    text = re_sub(r" r ", " are ")
    text = re_sub(r" y ", " why ")
    text = re_sub(r" Y ", " WHY ")
    text = re_sub(r"Y ", " WHY ")
    text = re_sub(r" hv ", " have ")
    text = re_sub(r" c ", " see ")
    text = re_sub(r" bcz ", " because ")
    text = re_sub(r" coz ", " because ")
    text = re_sub(r" v ", " we ")
    text = re_sub(r" ppl ", " people ")
    text = re_sub(r" pepl ", " people ")
    text = re_sub(r" r b i ", " rbi ")
    text = re_sub(r" R B I ", " RBI ")
    text = re_sub(r" R b i ", " rbi ")
    text = re_sub(r" R ", " ARE ")
    text = re_sub(r" hav ", " have ")
    text = re_sub(r"R ", " ARE ")
    text = re_sub(r" U ", " you ")
    text = re_sub(r" 👎 ", " em_negative ")
    text = re_sub(r"U ", " you ")
    text = re_sub(r" pls ", " please ")
    text = re_sub(r"Pls ", "Please ")
    text = re_sub(r"plz ", "please ")
    text = re_sub(r"Plz ", "Please ")
    text = re_sub(r"PLZ ", "Please ")
    text = re_sub(r"Pls", "Please ")
    text = re_sub(r"plz", "please ")
    text = re_sub(r"Plz", "Please ")
    text = re_sub(r"PLZ", "Please ")
    text = re_sub(r" thankz ", " thanks ")
    text = re_sub(r" thnx ", " thanks ")
    text = re_sub(r"fuck\w+ ", " fuck ")
    text = re_sub(r"f\*\* ", " fuck ")
    text = re_sub(r"\*\*\*k ", " fuck ")
    text = re_sub(r"F\*\* ", " fuck ")
    text = re_sub(r"mo\*\*\*\*\* ", " fucker ")
    text = re_sub(r"b\*\*\*\* ", " blody ")
    text = re_sub(r" mc ", " fucker ")
    text = re_sub(r" MC ", " fucker ")
    text = re_sub(r" wtf ", " fuck ")
    text = re_sub(r" ch\*\*\*ya ", " fucker ")
    text = re_sub(r" ch\*\*Tya ", " fucker ")
    text = re_sub(r" ch\*\*Tia ", " fucker ")
    text = re_sub(r" C\*\*\*yas ", " fucker ")
    text = re_sub(r"l\*\*\*\* ", "shit ")
    text = re_sub(r" A\*\*\*\*\*\*S", " ASSHOLES")
    text = re_sub(r" di\*\*\*\*s", " cker")
    text = re_sub(r" nd ", " and ")
    text = re_sub(r"Nd ", "and ")
    text = re_sub(r"([!?!]){2,}", r"! ")
    text = re_sub(r"([.?.]){2,}", r". ")
    text = re_sub(r"([*?*]){2,}", r"* ")
    text = re_sub(r"([,?,]){2,}", r", ")
    text = re_sub(r"ha?ha", r" em_positive ")
    # text = re_sub(r"([!]){2,}", r"! ")
    # text = re_sub(r"([.]){2,}", r". ")
    # text = re_sub(r"([*]){2,}", r"* ")
    # text = re_sub(r"([,]){2,}", r", ")
    # text = re_sub(r"\n\r", " ")
    text = re_sub(r"(ind[vs]pak)", " india versus pakistan ")
    text = re_sub(r"(pak[vs]ind)", " pakistan versus india ")
    text = re_sub(r"(indvsuae)", " india versus United Arab Emirates ")
    text = re_sub(r"[sS]hut[Dd]own[jnuJNU]", " shut down jnu ")
    # text = re_sub(r"ShutDownJNU", " shut down jnu ")
    # text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", " number ")
    # text = re_sub(r"(.)\1\1+", r"\1") # remove funnnnny --> funny
    return text

In [10]:
# Add the paths to the corpus. It should end in aclImdb/train
CORPUS_PATH = "C:\\Users\\morzm\\jup_txts\\corpus\\"
# Add the path to the embeddings. It should end in GoogleNews-vectors-negative300.bin.gz
GOOGLE_VECTORS = "C:\\Users\\morzm\\jup_txts\\GoogleNews-vectors-negative300.bin.gz"

In [11]:
# Preprocessing the data
dataset = pre_process_data(CORPUS_PATH)
dataset[0]

(0,
 'Congratulations Sharon!  @Lynden_Leigh: T minus 24 hrs I leave the corporate world for full #entrepreneurship, .. \n')

In [12]:
# Method to tokenise and vectorise all the training data

def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
#    expected = [] this line appears in the book, but it's not necessary here!
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except KeyError:
                pass # No matching token in the Google w2v vocab
        vectorized_data.append(sample_vecs)

    return vectorized_data

In [13]:
# Method to get the target labels
def collect_expected(dataset):
    """ Peel off the target values from the dataset """
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [25]:
# Loading the word2vec embeddings

word_vectors = KeyedVectors.load_word2vec_format(GOOGLE_VECTORS,
    binary=True, limit=400000)

In [26]:
# Loading the dataset
vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)

In [27]:
# Creating training and validation partitions

split_point = int(len(vectorized_data)*.8)
x_train = vectorized_data[:split_point] # there's a typo in this line, if copying from the book
y_train = expected[:split_point]
x_test = vectorized_data[split_point:]
y_test = expected[split_point:]

In [28]:
# Network parameters

maxlen = 70          # maximum length of the text (why?) --- number of vectors (word embeddings), playable
batch_size = 32       # number of samples before backpropagating, usually powers of 2, playable
embedding_dims = 300  # Same as Google's, don't play with this
filters = 250         # (!)
kernel_size = 3       # remember: filter=kernel (we have a scalar this time), you can play with this
hidden_dims = 250     # number of neurons in the final layer, playable
epochs = 3            # number of training epochs, playable



In [29]:
# Method to pad or truncate the input
# (notice that this code is quite verbose)
def pad_trunc(data, maxlen):
    """
    For a given dataset pad with zero vectors or truncate to maxlen
    """
    new_data = []
    # Create a vector of 0s the length of our word vectors
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)

    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            # Append the appropriate number 0 vectors to the list
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)

    return new_data

In [30]:
# Padding/truncating the data (if necessary)

x_train = pad_trunc(x_train, maxlen)
x_test = pad_trunc(x_test, maxlen)

# The shape is [number of samples, sequence length, word vector]  CUBE 
x_train = np.reshape(x_train, (len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), maxlen, embedding_dims))
y_test = np.array(y_test)

MemoryError: Unable to allocate 61.3 GiB for an array with shape (391920, 70, 300) and data type float64